In [1]:
MARKET = 'SNP'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = DATAPATH.joinpath('temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [6]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
files

['dfrq.pkl',
 'df_chains.pkl',
 'df_ohlcs.pkl',
 'df_opts.pkl',
 'df_opt_margins.pkl',
 'df_opt_prices.pkl',
 'df_symlots.pkl',
 'df_unds.pkl',
 'df_und_margins.pkl',
 'df_und_prices.pkl',
 'qopts.pkl']

# Refactoring Edge

In [7]:
from engine import qualify, executeAsync, margin, save_df, price
from support import Timer, quick_pf
from dfrq import dfrq

In [8]:
%%time
ibp = Vars(MARKET.upper())  # IB Parameters from var.yml

HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

LOGPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", "log")
DATAPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", MARKET.lower())

# * SETUP LOGS AND CLEAR THEM
LOGFILE = LOGPATH.joinpath(MARKET.lower() + "_dfrq.log")
util.logToFile(path=LOGFILE, level=30)
with open(LOGFILE, "w"):
    pass

# . start the time
edge_time = Timer('dfrqs')
edge_time.start()

# * PREPARE FILES AND GET dfrq
df_opts = pd.read_pickle(DATAPATH.joinpath('df_opts.pkl'))
dfrq = dfrq(MARKET)


Done margin for ['ETSY1113P118.0..', 'ETSY0.0..'] 96 out of 96. Pending []
Wall time: 5.54 s


In [9]:
# .edge symbols are `fresh` ones from dfrq
fresh = set(dfrq[dfrq.status == 'fresh'].symbol)

# * BUILD THE EDGE

# get underlying fresh contracts
und_cts = df_unds[df_unds.symbol.isin(fresh)].contract

In [10]:
%%time
# get price and volatility for these fresh contracts
with IB().connect(HOST, PORT, CID) as ib:
    df_piv = ib.run(executeAsync(ib, price, und_cts, save_df, CONCURRENT=40, TIMEOUT=8))


Done price for ['DJX0.0..', 'TAP0.0..'] 40 out of 207. Pending ['ZTS0.0..', 'ADBE0.0..']

Done price for ['GE0.0..', 'DPZ0.0..'] 80 out of 207. Pending ['KHC0.0..', 'ADP0.0..']

Done price for ['BAX0.0..', 'CAG0.0..'] 120 out of 207. Pending ['URI0.0..', 'EW0.0..']

Done price for ['DLTR0.0..', 'AMAT0.0..'] 160 out of 207. Pending ['HUM0.0..', 'KR0.0..']

Done price for ['GOOG0.0..', 'PHM0.0..'] 200 out of 207. Pending ['MRO0.0..', 'AZO0.0..']

Done price for ['MRO0.0..', 'MYL0.0..'] 207 out of 207. Pending []
Wall time: 33.5 s


In [11]:
df_piv

,symbol,secType,localSymbol,conId,strike,expiry,right,contract,time,model,bid,ask,close,last,price,iv
0,MOS,STK,MOS,88292752,0.0,,,"Contract(secType='STK', conId=88292752, symbol...",2020-11-13 10:34:10.955625+00:00,None,16.90,18.68,17.36,NaN,17.36,0.477355
1,VFC,STK,VFC,13440,0.0,,,"Contract(secType='STK', conId=13440, symbol='V...",2020-11-13 10:34:10.955625+00:00,None,74.67,78.07,76.16,NaN,76.16,0.363711
2,KLAC,STK,KLAC,270957,0.0,,,"Contract(secType='STK', conId=270957, symbol='...",2020-11-13 10:33:54.089099+00:00,None,234.66,238.55,232.36,235.13,235.13,0.377718
3,EOG,STK,EOG,6890,0.0,,,"Contract(secType='STK', conId=6890, symbol='EO...",2020-11-13 10:33:54.659220+00:00,None,39.63,41.00,40.49,NaN,40.49,0.554008
4,GILD,STK,GILD,269753,0.0,,,"Contract(secType='STK', conId=269753, symbol='...",2020-11-13 10:33:53.843765+00:00,None,59.93,60.29,59.98,NaN,59.98,0.272335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,OKE,STK,OKE,10794,0.0,,,"Contract(secType='STK', conId=10794, symbol='O...",2020-11-13 10:34:00.015036+00:00,None,30.26,31.22,30.77,NaN,30.77,0.502700
203,SPGI,STK,SPGI,229629397,0.0,,,"Contract(secType='STK', conId=229629397, symbo...",2020-11-13 10:34:00.180268+00:00,None,335.83,349.49,341.72,NaN,341.72,0.275060
204,NOC,STK,NOC,10376,0.0,,,"Contract(secType='STK', conId=10376, symbol='N...",2020-11-13 10:34:17.103930+00:00,None,301.74,311.60,306.23,NaN,306.23,0.258259
205,VRTX,STK,VRTX,275850,0.0,,,"Contract(secType='STK', conId=275850, symbol='...",2020-11-13 10:34:00.174287+00:00,None,218.16,221.31,218.31,NaN,218.31,0.309227


In [16]:
df_piv.secType.unique()

array(['STK', 'IND'], dtype=object)

In [14]:
207 + len(dfrq[dfrq.status != 'fresh'])

261